### Prepare space in flash

In [1]:
import os, subprocess

In [2]:
rootdir = "/bucket/ReiterU/DBS/data/20220529_P191_laqueus_DiI_LM13/"
resultdir = os.path.join(rootdir, "analysis")
if not os.path.exists(resultdir):
    subprocess.run(["ssh", "deigo", "mkdir", resultdir])

In [3]:
cwd = os.path.basename(os.getcwd())
tmpdir = os.path.join("/flash/ReiterU/tmp/", cwd, "ants")
if not os.path.exists(tmpdir):
    os.makedirs(tmpdir)

### Prepare mask

In [4]:
import nrrd
from probemapper.io import save_nifti, load_nifti
import numpy as np

In [5]:
raw = load_nifti(os.path.join(rootdir, "analysis/536BP_10um.nii"))
mask, _ = nrrd.read(os.path.join(rootdir, "analysis/slicer_536BP/Segmentation.seg.nrrd"))
mask = np.swapaxes(mask, 0, 2)
raw[mask==0] = 0
save_nifti(raw, os.path.join(tmpdir, "536BP_10um_masked.nii"), 10, 10, 10)

In [6]:
raw = load_nifti("/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/SYTOXG_10um.nii")
mask = load_nifti("/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/SYTOXG_10um_mask.nii.gz")
raw[mask==0] = 0
save_nifti(raw, os.path.join(tmpdir, "O_Laqueues_v0.1_10um_masked.nii"), 10, 10, 10)

### Registration

In [7]:
from probemapper.registration import ANTSParams, ANTSStage, ANTSRegisteration

In [8]:
from probemapper.registration.helper import define_initial_transformation
import numpy as np

aff = np.array([[ 1.0, 0, 0],
                [ 0, 1.0, 0],
                [ 0, 0, 1.0]], dtype='float')
transl = np.array([0, 0, 0], dtype='float')
define_initial_transformation(aff, transl, os.path.join(tmpdir, "initial.mat"))

.mat file saved as /flash/ReiterU/tmp/P191/ants/initial.mat


In [9]:
P = ANTSParams(
    use_float=False,
    interpolation="Linear",
    histogram_matching=False,
    winsorize=[0.05, 1.0],
    init_transform_fx_to_mv=os.path.join(tmpdir, "initial.mat"),
    stages=[
        ANTSStage(
            transform="Affine",
            transform_args="0.1",
            metrics="MI",
            metrics_args="1,128,Regular,0.5",
            convergence="1000x1000x1000,1e-5,15",
            shrink_factors="8x4x2",
            smoothing_sigmas="3x2x1vox"
        ),
        ANTSStage(
            transform="SyN",
            transform_args="0.1,4.0,0.2",
            metrics="CC",
            metrics_args="1,4",
            convergence="500x500x100x30,1e-6,10",
            shrink_factors="8x4x2x1",
            smoothing_sigmas="3x2x1x0vox"
        )
    ]
)

In [10]:
R = ANTSRegisteration(P, ants_path="/apps/unit/ReiterU/ANTs/2.3.5/bin", num_threads=32)
R.set_fixed_image(os.path.join(tmpdir, "O_Laqueues_v0.1_10um_masked.nii"), 10)
R.set_moving_image(os.path.join(tmpdir, "536BP_10um_masked.nii"), 10)
R.set_outputs(tmpdir)

In [11]:
R.run()

########################################
['/apps/unit/ReiterU/ANTs/2.3.5/bin/antsRegistration', '--dimensionality', '3', '--initial-moving-transform', '/flash/ReiterU/tmp/P191/ants/initial.mat', '--interpolation', 'Linear', '--use-histogram-matching', '0', '--float', '0', '--verbose', '1', '--output', '[/flash/ReiterU/tmp/P191/ants/F2M_,/flash/ReiterU/tmp/P191/ants/Warped.nii.gz,/flash/ReiterU/tmp/P191/ants/InvWarped.nii.gz]', '--transform', 'Affine[0.1]', '--metric', 'MI[/flash/ReiterU/tmp/P191/ants/O_Laqueues_v0.1_10um_masked.nii,/flash/ReiterU/tmp/P191/ants/536BP_10um_masked.nii,1,128,Regular,0.5]', '--convergence', '[1000x1000x1000,1e-5,15]', '--shrink-factors', '8x4x2', '--smoothing-sigmas', '3x2x1vox', '--transform', 'SyN[0.1,4.0,0.2]', '--metric', 'CC[/flash/ReiterU/tmp/P191/ants/O_Laqueues_v0.1_10um_masked.nii,/flash/ReiterU/tmp/P191/ants/536BP_10um_masked.nii,1,6]', '--convergence', '[500x500x100x30,1e-6,10]', '--shrink-factors', '8x4x2x1', '--smoothing-sigmas', '3x2x1x0vox']


 2DIAGNOSTIC,     6, -9.540272472325e-02, 1.797693134862e+308, 1.5345e+00, 4.2351e-02,
 2DIAGNOSTIC,     7, -9.557680462589e-02, 1.797693134862e+308, 1.5754e+00, 4.0916e-02,
 2DIAGNOSTIC,     8, -9.604651360446e-02, 1.797693134862e+308, 1.6183e+00, 4.2901e-02,
 2DIAGNOSTIC,     9, -9.629550300448e-02, 1.797693134862e+308, 1.6589e+00, 4.0644e-02,
 2DIAGNOSTIC,    10, -9.811094092750e-02, 1.797693134862e+308, 1.7024e+00, 4.3432e-02,
 2DIAGNOSTIC,    11, -9.980900842749e-02, 1.797693134862e+308, 1.7430e+00, 4.0664e-02,
 2DIAGNOSTIC,    12, -1.010984300173e-01, 1.797693134862e+308, 1.7857e+00, 4.2679e-02,
 2DIAGNOSTIC,    13, -1.070474739028e-01, 1.797693134862e+308, 1.8292e+00, 4.3508e-02,
 2DIAGNOSTIC,    14, -1.266935920584e-01, 1.797693134862e+308, 1.8767e+00, 4.7478e-02,
 2DIAGNOSTIC,    15, -1.451132213194e-01, 1.225208730019e-02, 1.9219e+00, 4.5242e-02,
 2DIAGNOSTIC,    16, -1.697596748723e-01, 1.822301959355e-02, 1.9650e+00, 4.3055e-02,
 2DIAGNOSTIC,    17, -3.021776734406e-01, 3.3

 2DIAGNOSTIC,   104, -4.028931526081e-01, 2.346517175926e-05, 6.4652e+00, 4.5985e-02,
 2DIAGNOSTIC,   105, -4.033983516047e-01, 2.601561661853e-05, 6.5170e+00, 5.1836e-02,
 2DIAGNOSTIC,   106, -4.052127829322e-01, 3.689966521813e-05, 6.5827e+00, 6.5688e-02,
 2DIAGNOSTIC,   107, -4.058822595261e-01, 4.960134467029e-05, 6.6331e+00, 5.0410e-02,
 2DIAGNOSTIC,   108, -4.054307922088e-01, 5.754582518023e-05, 6.7072e+00, 7.4070e-02,
 2DIAGNOSTIC,   109, -4.055901302627e-01, 6.372990369054e-05, 6.7574e+00, 5.0186e-02,
 2DIAGNOSTIC,   110, -4.056205427430e-01, 6.755651050257e-05, 6.8027e+00, 4.5323e-02,
 2DIAGNOSTIC,   111, -4.056432889742e-01, 6.920311056204e-05, 6.8460e+00, 4.3329e-02,
 2DIAGNOSTIC,   112, -4.057327734537e-01, 6.846987944662e-05, 6.8986e+00, 5.2585e-02,
 2DIAGNOSTIC,   113, -4.057653268780e-01, 6.613548564512e-05, 6.9627e+00, 6.4103e-02,
 2DIAGNOSTIC,   114, -4.057599734195e-01, 6.244454687434e-05, 7.1331e+00, 1.7037e-01,
 2DIAGNOSTIC,   115, -4.057599733579e-01, 5.6348866449

 2DIAGNOSTIC,    80, -4.736687589188e-01, 7.069461766456e-04, 2.5191e+01, 2.0345e-01,
 2DIAGNOSTIC,    81, -4.758531636885e-01, 6.831596837476e-04, 2.5365e+01, 1.7452e-01,
 2DIAGNOSTIC,    82, -4.770252286121e-01, 6.399416910608e-04, 2.5543e+01, 1.7748e-01,
 2DIAGNOSTIC,    83, -4.786217394025e-01, 5.850106999520e-04, 2.5717e+01, 1.7364e-01,
 2DIAGNOSTIC,    84, -4.872868019598e-01, 5.616488818264e-04, 2.5923e+01, 2.0599e-01,
 2DIAGNOSTIC,    85, -5.358989157216e-01, 8.469379240877e-04, 2.6258e+01, 3.3574e-01,
 2DIAGNOSTIC,    86, -5.535928247308e-01, 1.185717571576e-03, 2.6457e+01, 1.9881e-01,
 2DIAGNOSTIC,    87, -5.543710148400e-01, 1.468027460710e-03, 2.6719e+01, 2.6184e-01,
 2DIAGNOSTIC,    88, -5.545344419488e-01, 1.685719948868e-03, 2.6895e+01, 1.7608e-01,
 2DIAGNOSTIC,    89, -5.549697750649e-01, 1.835306608877e-03, 2.7127e+01, 2.3188e-01,
 2DIAGNOSTIC,    90, -5.550146593169e-01, 1.925498857519e-03, 2.7386e+01, 2.5874e-01,
 2DIAGNOSTIC,    91, -5.550254122924e-01, 1.9661267944

 1DIAGNOSTIC,    40, -8.221975076600e-01, 2.523317963258e-04, 4.5050e+01, 9.8661e-01,
 1DIAGNOSTIC,    41, -8.234304452147e-01, 2.538376461082e-04, 4.6052e+01, 1.0020e+00,
 1DIAGNOSTIC,    42, -8.245421250292e-01, 2.507024184991e-04, 4.7216e+01, 1.1642e+00,
 1DIAGNOSTIC,    43, -8.255945126020e-01, 2.421401494840e-04, 4.8338e+01, 1.1220e+00,
 1DIAGNOSTIC,    44, -8.264500185987e-01, 2.269161104067e-04, 4.9395e+01, 1.0564e+00,
 1DIAGNOSTIC,    45, -8.272226897934e-01, 2.066227624127e-04, 5.0452e+01, 1.0572e+00,
 1DIAGNOSTIC,    46, -8.279684314971e-01, 1.836205675506e-04, 5.1558e+01, 1.1065e+00,
 1DIAGNOSTIC,    47, -8.286462541934e-01, 1.605248350767e-04, 5.2705e+01, 1.1470e+00,
 1DIAGNOSTIC,    48, -8.292770147232e-01, 1.398119160388e-04, 5.3802e+01, 1.0970e+00,
 1DIAGNOSTIC,    49, -8.299374075571e-01, 1.239897523060e-04, 5.4946e+01, 1.1434e+00,
 1DIAGNOSTIC,    50, -8.305229666197e-01, 1.112681472881e-04, 5.6153e+01, 1.2072e+00,
 1DIAGNOSTIC,    51, -8.311050517062e-01, 1.0096849023

 1DIAGNOSTIC,   136, -8.499577043213e-01, 7.164152250402e-06, 1.5016e+02, 1.0634e+00,
 1DIAGNOSTIC,   137, -8.501048574069e-01, 6.535356895905e-06, 1.5125e+02, 1.0905e+00,
 1DIAGNOSTIC,   138, -8.501910669841e-01, 6.092959138738e-06, 1.5229e+02, 1.0434e+00,
 1DIAGNOSTIC,   139, -8.503230580090e-01, 5.826426012929e-06, 1.5330e+02, 1.0099e+00,
 1DIAGNOSTIC,   140, -8.504640586699e-01, 5.320378671405e-06, 1.5430e+02, 1.0010e+00,
 1DIAGNOSTIC,   141, -8.504924333909e-01, 5.002512436167e-06, 1.5536e+02, 1.0620e+00,
 1DIAGNOSTIC,   142, -8.508037254535e-01, 5.182747733723e-06, 1.5649e+02, 1.1296e+00,
 1DIAGNOSTIC,   143, -8.505619417524e-01, 5.187935577171e-06, 1.5758e+02, 1.0914e+00,
 1DIAGNOSTIC,   144, -8.507789620294e-01, 5.039446375190e-06, 1.5869e+02, 1.1070e+00,
 1DIAGNOSTIC,   145, -8.506827843595e-01, 4.493779777334e-06, 1.5986e+02, 1.1660e+00,
 1DIAGNOSTIC,   146, -8.508747240654e-01, 4.023414693755e-06, 1.6098e+02, 1.1199e+00,
 1DIAGNOSTIC,   147, -8.508827720387e-01, 3.5384494156

 1DIAGNOSTIC,    54, -8.831302397374e-01, 7.894247151460e-06, 5.5795e+02, 6.8168e+00,
 1DIAGNOSTIC,    55, -8.831600196536e-01, 7.512970623222e-06, 5.6458e+02, 6.6274e+00,
 1DIAGNOSTIC,    56, -8.831893501502e-01, 6.968831117258e-06, 5.7133e+02, 6.7476e+00,
 1DIAGNOSTIC,    57, -8.832738051429e-01, 6.583147211119e-06, 5.7794e+02, 6.6133e+00,
 1DIAGNOSTIC,    58, -8.832994465014e-01, 6.335786011759e-06, 5.8462e+02, 6.6821e+00,
 1DIAGNOSTIC,    59, -8.833239465153e-01, 5.923541844505e-06, 5.9141e+02, 6.7884e+00,
 1DIAGNOSTIC,    60, -8.834001893872e-01, 5.640097865873e-06, 5.9790e+02, 6.4842e+00,
 1DIAGNOSTIC,    61, -8.834239117161e-01, 5.487070108074e-06, 6.0460e+02, 6.7067e+00,
 1DIAGNOSTIC,    62, -8.834448381246e-01, 5.158151783539e-06, 6.1110e+02, 6.4984e+00,
 1DIAGNOSTIC,    63, -8.835170743700e-01, 4.939329376723e-06, 6.1776e+02, 6.6621e+00,
 1DIAGNOSTIC,    64, -8.835350364708e-01, 4.804698927888e-06, 6.2435e+02, 6.5844e+00,
 1DIAGNOSTIC,    65, -8.835494740942e-01, 4.4787205150

 1DIAGNOSTIC,    42, -8.886826113552e-01, 2.638744778743e-05, 3.0886e+03, 5.2058e+01,
 1DIAGNOSTIC,    43, -8.888412172501e-01, 2.578639849942e-05, 3.1410e+03, 5.2327e+01,
 1DIAGNOSTIC,    44, -8.889917007179e-01, 2.509341322045e-05, 3.1933e+03, 5.2318e+01,
 1DIAGNOSTIC,    45, -8.891388823493e-01, 2.431383866007e-05, 3.2457e+03, 5.2469e+01,
 1DIAGNOSTIC,    46, -8.892782505564e-01, 2.339163797606e-05, 3.2973e+03, 5.1562e+01,
 1DIAGNOSTIC,    47, -8.894137183653e-01, 2.234450591155e-05, 3.3498e+03, 5.2483e+01,
 1DIAGNOSTIC,    48, -8.895450544639e-01, 2.123051598543e-05, 3.4018e+03, 5.2040e+01,
 1DIAGNOSTIC,    49, -8.896713636038e-01, 2.008424572533e-05, 3.4532e+03, 5.1348e+01,
 1DIAGNOSTIC,    50, -8.897983075535e-01, 1.900190862115e-05, 3.5043e+03, 5.1114e+01,
 1DIAGNOSTIC,    51, -8.899186898162e-01, 1.801922650998e-05, 3.5554e+03, 5.1147e+01,
 1DIAGNOSTIC,    52, -8.900373306038e-01, 1.713187960573e-05, 3.6067e+03, 5.1280e+01,
 1DIAGNOSTIC,    53, -8.901513364407e-01, 1.6324811877

### Copy files to bucket

In [12]:
subprocess.run(["scp", "-r", tmpdir, f"deigo:{resultdir}"])

CompletedProcess(args=['scp', '-r', '/flash/ReiterU/tmp/P191/ants', 'deigo:/bucket/ReiterU/DBS/data/20220529_P191_laqueus_DiI_LM13/analysis'], returncode=0)